In [3]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 

In [8]:
class TwitterClient(object): 
    ''' 
    Generic Twitter Class for sentiment analysis. 
    '''
    def __init__(self): 
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        #consumer_key="Ev93iW0STr4EymlrqiImSE9yx"
        consumer_key = "aHtGWhDgKtBTfohXBe9gMdUgh"

        #consumer_secret="xXwORVl6WDjjAIZqhfca4xswBuLs4JQ4EIsWHdfwLay3wdQa5j"
        consumer_secret="pSpOjgCvKcoW9ZmLtfrzKk4JpA7604VLlA0N5Thz2CvPEMk344"

        #access_token="1037717238525190144-m88Se0GNRFoRb1o00YybdOTXlKCLMG"
        access_token="710845973400915968-rexTM2eE79GX0PvX10Ervme2pbMTXFE"

        #access_token_secret="u5HI3J7ENvk7oOughDKZQKziBvxIg3NZ0RSSalG2HZMW4"
        access_token_secret="9jC9udCr8GAXqLzvA8WAKJG8VvchAVl8HQWziF3oaIRq8"
        
        
  
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub(u"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
  
    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 

In [13]:
  
def main(): 
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    # calling function to get tweets 
    tweets = api.get_tweets(query = 'Donald Trump', count = 200) 
    
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    # percentage of neutral tweets 
    print("Neutral tweets percentage: {} % ".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets))) 
  
    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:10]: 
        print(tweet['text']) 
  
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:10]: 
        print(tweet['text']) 
  
if __name__ == "__main__": 
    # calling main function 
    main() 

Positive tweets percentage: 41.891891891891895 %
Negative tweets percentage: 18.91891891891892 %
Neutral tweets percentage: 39.189189189189186 % 


Positive tweets:
RT @PalmerReport: Confirmed: Rod Rosenstein has NOT been fired, and this was all a distraction by Donald Trump https://t.co/6j9rrvNnwq
@ShrewingTheTame @ANN13951880 @samstein @rhonda_harbison I believe most men would have already stepped away citing… https://t.co/hF1UwLYjDq
US: https://t.co/Zjqp1Hf0Gj
UK: https://t.co/JDZOxltOVj
CANADA: https://t.co/svlO1PmKHz
READ FREE on… https://t.co/SWsS3EVo6d
RT @PatCunnane: Donald Trump gets all the attention, but no one has done more behind the scenes than Mitch McConnell to undermine our insti…
RT @CARandDRIVER: "The Beast," @realDonaldTrump's new @Cadillac presidential limo, has arrived: https://t.co/JG1uaBzHxs https://t.co/5vXgRI…
@HoarseWisperer WE SHOULD ALL BE MAKING THE FACT THAT DONALD TRUMP IS USING HIS TITLE TO SELF-ENRICH HIMSELF A MUCH… https://t.co/JO7qHuwC9m
RT @inittow